In [1]:
import pandas as pd
import re
import os
import sys
import time
import numpy as np

In [2]:
path = '/Users/gongchenyin/Desktop/资料馆/科研/公司实验/src/数据资料/参考和模板/BQD/baseline question data test.xlsx'

In [3]:
bqd = pd.read_excel(path)

In [4]:
bqd[["填写答案1"]].to_csv('/Users/gongchenyin/PycharmProjects/ra/BQData-Formatting/src/temp')

## 一、中文数字转换成阿拉伯数字

In [5]:
cn_num = {
    '〇' : 0, '一' : 1, '二' : 2, '三' : 3, '四' : 4, '五' : 5, '六' : 6, '七' : 7, '八' : 8, '九' : 9, '零' : 0, 
    '壹' : 1, '贰' : 2, '叁' : 3, '肆' : 4, '伍' : 5, '陆' : 6, '柒' : 7, '捌' : 8, '玖' : 9, '貮' : 2, '两' : 2,
}

In [6]:
cn_unit = {
    '十' : 10,
    '拾' : 10,
    '百' : 100,
    '佰' : 100,
    '千' : 1000,
    '仟' : 1000,
    '万' : 10000,
    '萬' : 10000,
    '亿' : 100000000,
    '億' : 100000000,
    '兆' : 1000000000000,
}

In [7]:
def chinese_to_arabic(cn:str) -> int:
    if cn == '':
        return np.nan
    
    if cn[0] in cn_unit and cn[0] not in ["十", "拾"]:
        return np.nan
    
    
    if cn[-1] in cn_num and len(cn) > 1:
        for i in cn[::-1][1:]:
            if i == "零" or i == "〇":
                break
            elif i in cn_unit and i not in ["十", "拾"]:
                return np.nan
            else:
                break
    
    unit = 0
    ldig = []
    last_is_num = False
    last_is_unit = False
    last_letter = ''
    for cndig in reversed(cn):
        if cndig in cn_unit:
            if last_is_unit and (last_letter not in ["万", "萬", "亿", "億", "兆"]):
                return np.nan
            last_is_unit = True
            last_is_num = False
            unit = cn_unit.get(cndig)
            if unit == 10000 or unit == 100000000:
                last_letter = cndig
                ldig.append(unit)
                unit = 1
        else:
            if last_is_num and cndig not in ["零", "〇"]:
                return np.nan
            last_is_num = True
            last_is_unit = False
            dig = cn_num.get(cndig)
            if unit:
                dig *= unit
                unit = 0
            ldig.append(dig)
    if unit == 10:
        ldig.append(10)
    val, tmp = 0, 0
    for x in reversed(ldig):
        if x == 10000 or x == 100000000:
            val += tmp * x
            tmp = 0
        else:
            tmp += x
    val += tmp
    return val

In [8]:
def test1():
    test_dig = ["零",
               "零一",
               "二十万",
               "两千",
               "两千一",    
               "两千十二",  
               "十八",   
               "一万五",   
               "一万五百",
               "一万千",
               "三百三十",
               "三百零三",
               "三千零一十一",
               "十百",
               "十",
               "万",
               "十万"
               "十百八",
               '',
               "三五万"]
    for cn in test_dig:
        x = chinese_to_arabic(cn)
        print(cn, x)
test1()

零 0
零一 1
二十万 200000
两千 2000
两千一 nan
两千十二 nan
十八 18
一万五 nan
一万五百 10500
一万千 nan
三百三十 330
三百零三 303
三千零一十一 3011
十百 nan
十 10
万 nan
十万十百八 nan
 nan
三五万 nan


## 二、阿拉伯数字转换成中文数字

In [9]:
mapping = ['零', '一', '二', '三', '四', '五', '六', '七', '八', '九', '十', 
           '十一', '十二', '十三', '十四', '十五', '十六', '十七', '十八', '十九']
p0 = ['', '十', '百', '千']
s4 = 10 ** 4

In [10]:
def arabic_to_chinese(num:int) -> str:
    if (num < 0 or num >= s4):
        return ''
    if num < 20:
        return mapping[num]
    else:
        lst = []
        while num >= 10:
            lst.append(num % 10)
            num = num // 10
        lst.append(num)
        c = len(lst)  # 位数
        result = ''
        for idx, val in enumerate(lst):
            val = int(val)
            if val != 0:
                result += p0[idx] + mapping[val]
                if (idx < c-1) and (lst[idx+1] == 0):
                    result += '零'
        return result[::-1]

In [11]:
def test2():
    test_dig = [0,
               20,
               1234,
               3011,
               301,
               310,
               1001,
               10000,
               -1]
    for num in test_dig:
        x = arabic_to_chinese(num)
        y = chinese_to_arabic(x)
        print(num, x, y)

In [12]:
test2()

0 零 0
20 二十 20
1234 一千二百三十四 1234
3011 三千零一十一 3011
301 三百零一 301
310 三百一十 310
1001 一千零一 1001
10000  nan
-1  nan


## 三、阿拉伯+中文混合数字转换成阿拉伯数字
### 例如：100万、20万、5万、2千、3百

In [13]:
import re

In [14]:
# isdigit: 如果字符串只包含数字则返回 True 否则返回 False
'6'.isdigit()

True

In [15]:
print(re.split(r'(\d+)', "六万"))
print(re.split(r'(\d+)', "60000"))
print(re.split(r'(\d+)', "6万"))
print(re.split(r'(\d+)', "6万3千"))
print(re.split(r'(\d+)', "6万多"))
print(re.split(r'(\d+)', "60多万"))
print(re.split(r'(\d+)', "6万以上"))
print(re.split(r'(\d+)', "大概6万"))
print(re.split(r'(\d+)', "60几万"))
print(re.split(r'(\d+)', "5万到16万"))
print(re.split(r'(\d+)', "1号30万"))

['六万']
['', '60000', '']
['', '6', '万']
['', '6', '万', '3', '千']
['', '6', '万多']
['', '60', '多万']
['', '6', '万以上']
['大概', '6', '万']
['', '60', '几万']
['', '5', '万到', '16', '万']
['', '1', '号', '30', '万']


### 先考虑如下：
#### 60万、60000、六十万、6万3千、6千、6百、3千3

In [88]:
def hybrid_to_arabic(hy:str) -> int:
    if hy == '':
        return np.nan
    elif hy.isdigit():
        if int(hy) >= 100000000000000000000:
            return np.nan
        return int(hy)
    else:
        lst = re.split(r'(\d+)', hy)
        for i, s in enumerate(lst):
            if s.isdigit() and 0 <= int(s) < 10**4:
                lst[i] = arabic_to_chinese(int(s))
        return chinese_to_arabic(''.join(lst))

In [17]:
def test3():
    test_dig = ['600000',
                '100000000000000000000',
               '60万',
               '六十万',
               '陆拾万',
               '6万3千',
               '6千',
               '6百',
               '1245亿50万',
               '3千3',        # 3千3需要改
               '',
               '10万',
               '七八万', # 有问题
               '十八',    # ?
               '八八', # ？
               '十万万',
               '九2万', # ？
               '1万'
               ]       
    for num in test_dig:
        x = hybrid_to_arabic(num)
        print(num, x)

In [18]:
test3()

600000 600000
100000000000000000000 nan
60万 600000
六十万 600000
陆拾万 600000
6万3千 63000
6千 6000
6百 600
1245亿50万 124500500000
3千3 nan
 nan
10万 100000
七八万 nan
十八 18
八八 nan
十万万 100000
九2万 nan
1万 10000


## 四、去除特殊和修饰字符
### 标点符号、空格、“大概、将近、以内、以上、以下、左右、多、几”等等

### 去除大概、将近、以内、以上、以下、左右、多、几等等修饰语
万：有效，但缺少数字，无效 ✅

6: 小数字，但缺少万，改为6万

将近6万：无效+有效 ✅

几十万：无效+有效，但无效

一万5: 需要转换成一万五千，而不是一万零五

6万多：有效+无效 ✅

六万多 ✅

6万左右 ✅

10多万：有效+无效+有效 ✅

10几万 ✅

大概10万左右：无效+有效+无效 ✅

3万到5万：有效 + 无效 + 有效，取最后一个有效 ✅

三到五万 ✅

四五万

4到5万 ✅

6 7万 ✅

7，8万 ✅

In [19]:
valid_num = [
    '〇', '一', '二', '三', '四', '五', '六', '七', '八', '九', '零', 
    '壹', '贰', '叁', '肆', '伍', '陆', '柒', '捌', '玖', '貮', '两',
    '十', '拾'
]

valid_unit = [
    '百', '佰', '千', '仟', '万', '萬', '亿', '億', '兆'
]

In [20]:
re1 = '[^' + ''.join(valid_num) + ''.join(valid_unit) + '\d]+'
re1

'[^〇一二三四五六七八九零壹贰叁肆伍陆柒捌玖貮两十拾百佰千仟万萬亿億兆\\d]+'

In [21]:
reObj1 = re.compile(re1)

In [22]:
valid_char = [
    '来', '多', '几', '左', '吧', '以', '之', '内'
]

In [23]:
re2a = '[0-9' + ''.join(valid_num) + ']+'
re2b = '[^0-9' + ''.join(valid_num) + ''.join(valid_unit) + ''.join(valid_char) + '元 ]+'
re2 = re2a + re2b
re2

'[0-9〇一二三四五六七八九零壹贰叁肆伍陆柒捌玖貮两十拾]+[^0-9〇一二三四五六七八九零壹贰叁肆伍陆柒捌玖貮两十拾百佰千仟万萬亿億兆来多几左吧以之内元 ]+'

In [24]:
reObj2 = re.compile(re2)

In [25]:
if reObj2.search("134元"):
    print('符合')
else:
    print('不符合')

不符合


In [129]:
re3a = '[0-9' + ''.join(valid_num) + ''.join(valid_unit) + ']+'
re3b = '[^0-9' + ''.join(valid_num) + ''.join(valid_unit) + ']+'
# re3b = '[^0-9' + ''.join(valid_num) + ''.join(valid_unit) + '来几元]+'
re3c = '[0-9' + ''.join(valid_num) + ']+'
re3d = '[' + ''.join(valid_unit) + ']*'
re3 = re3a + re3b + re3c + re3d
re3

'[0-9〇一二三四五六七八九零壹贰叁肆伍陆柒捌玖貮两十拾百佰千仟万萬亿億兆]+[^0-9〇一二三四五六七八九零壹贰叁肆伍陆柒捌玖貮两十拾百佰千仟万萬亿億兆]+[0-9〇一二三四五六七八九零壹贰叁肆伍陆柒捌玖貮两十拾]+[百佰千仟万萬亿億兆]*'

In [130]:
reObj3 = re.compile(re3)

In [133]:
if reObj3.search("10 万吧"):
    print('符合')
else:
    print('不符合')

不符合


In [29]:
# 将字符串按数字、”万“、其他字符拆分为list
def robust_clean(num) -> str:
    # 先将num转化为str类型
    num = str(num)
    # 忽略多次出现数字字符的情形（空格也算分割点），如“1号113万”、“118. 25万”、”200 0万“、“2o万”
    if reObj3.search(num):
        return ''
    # 忽略数字之后跟随非有效词缀的词/字，如“100号”，“一般”
    if reObj2.search(num):
        return ''
    result = reObj1.sub('', num)
    return result

In [30]:
def test4():
    test_dig = ['一万5',
                '100',
                '20 20',
                '2o万',
                '2万 3',
                '30 万',
                '分担一些',
                '0鸭蛋',
                '五位数',
                '222. 10号',
                '一万，',
                '几十万',
                "10万多，将近11万",
                "10万，将近11万",
                "三四万"
               ]       
    for num in test_dig:
        x = robust_clean(num)
        print(num + ':  ' +  x)
test4()

一万5:  一万5
100:  100
20 20:  
2o万:  
2万 3:  
30 万:  30万
分担一些:  
0鸭蛋:  
五位数:  
222. 10号:  
一万，:  一万
几十万:  十万
10万多，将近11万:  
10万，将近11万:  
三四万:  三四万


In [31]:
bqd[["填写答案1"]].applymap(robust_clean).to_csv('/Users/gongchenyin/PycharmProjects/ra/BQData-Formatting/src/temp2')

## 五、统一数字表达为阿拉伯数字

In [32]:
bqd[["填写答案1"]].applymap(robust_clean).applymap(hybrid_to_arabic).to_csv('/Users/gongchenyin/PycharmProjects/ra/BQData-Formatting/src/temp3')

In [33]:
robust_clean("2o万")

''

In [34]:
df3 = bqd[["填写答案1"]].applymap(robust_clean).applymap(hybrid_to_arabic)

In [35]:
len(df3[np.isnan(df3["填写答案1"])])/len(df3)

0.2225

## 测试

In [134]:
s1 = '3 万'
s2 = robust_clean(s1)
s3 = hybrid_to_arabic(s2)
print('最开始：', s1)
print('清洗后：', s2)
print('转换后：', s3)

三万
最开始： 3 万
清洗后： 3万
转换后： 30000


## 六、文件读取

In [38]:
file_path = '/Users/gongchenyin/PycharmProjects/ra/data/201907/BQD_v1_test/172.xlsx'

In [39]:
read_df = pd.read_excel(file_path, sheet_name=0, header=None, skiprows=1)

In [40]:
# 删除后面多余的空值列
read_df.dropna(axis=1, how='all', inplace=True)

In [41]:
read_df.head(2)

,0,1,2,3,4,5,6,7
0,部门,姓名,编号,填写时间,问题1,NaN,问题2,NaN
1,NaN,NaN,NaN,NaN,问题,填写答案数字,问题,填写答案


In [42]:
# 将0、1两行的nan值填充为’‘，第1行的“问题”字符改为’‘，再将两列相加
read_df.fillna('', inplace=True)

In [43]:
read_df.head(2)

,0,1,2,3,4,5,6,7
0,部门,姓名,编号,填写时间,问题1,,问题2,
1,,,,,问题,填写答案数字,问题,填写答案


In [44]:
row0 = read_df.iloc[0]
row1 = read_df.iloc[1]

In [45]:
row1.iloc[row1.values == '问题'] = ''

In [46]:
header = row0 + row1

In [47]:
# 将合并后的行作为变量名行插入
read_df.drop([0,1], inplace=True)

In [48]:
read_df.reset_index(drop=True, inplace=True)

In [49]:
read_df.columns = header

In [50]:
"填写答案数字" in header.values

True

In [51]:
read_df.head()

,部门,姓名,编号,填写时间,问题1,填写答案数字,问题2,填写答案
0,北京鸟巢店,张学成,121110002,2019-05-22 00:00:00,您认为您所在门店上个月业绩最高的员工业绩(销售额)是多少,不知道,"您对您的答案有多确定\r\n选项：A. 很确定,B. 比较确定,C. 一般,D. 不太确定,...",C. 一般
1,北京通州旗舰店,袁士兰,160505005,2019-05-22 00:00:00,您认为您所在门店上个月业绩最高的员工业绩(销售额)是多少,20万,"您对您的答案有多确定\r\n选项：A. 很确定,B. 比较确定,C. 一般,D. 不太确定,...",D. 不太确定
2,北京万寿路店,杨建英,170228057,2019-05-22 00:00:00,您认为您所在门店上个月业绩最高的员工业绩(销售额)是多少,30,"您对您的答案有多确定\r\n选项：A. 很确定,B. 比较确定,C. 一般,D. 不太确定,...",B. 比较确定
3,北京万寿路店,闫江梅,170301006,2019-05-22 00:00:00,您认为您所在门店上个月业绩最高的员工业绩(销售额)是多少,80万,"您对您的答案有多确定\r\n选项：A. 很确定,B. 比较确定,C. 一般,D. 不太确定,...",A. 很确定
4,北京华彩店,杜瑞英,17030719,2019-05-22 00:00:00,您认为您所在门店上个月业绩最高的员工业绩(销售额)是多少,十万,"您对您的答案有多确定\r\n选项：A. 很确定,B. 比较确定,C. 一般,D. 不太确定,...",C. 一般


In [52]:
# 获取“填写答案数字”的列索引值
index_of_answer = 0
for i in range(len(read_df.columns)):
    if read_df.columns[i] == "填写答案数字":
        index_of_answer = i

In [53]:
read_df.insert(index_of_answer, "填写答案数字（改）", read_df[["填写答案数字"]].applymap(robust_clean).applymap(hybrid_to_arabic))

In [54]:
read_df.sort_values(by="填写答案数字（改）", inplace=True)
read_df.reset_index(drop=True, inplace=True)

In [55]:
# 判断答案中是否包含“不清楚”、“不知道”等语句
def unknown(answer: str) -> bool:
    words = ["不清楚", "不太清楚", "不知道", "不确定", "不太确定", "未知"]
    find = False
    for word in words:
        if answer.find(word) != -1:
            find = True
    return find

In [56]:
read_df.insert(index_of_answer, "填写答案数字（改+）", read_df[["填写答案数字（改）"]])
read_df['排序标识符'] = 2

In [57]:
# 插入另一列，将小于100的数字改为100,000；同时添加排序标识符，用于排序
for i in range(len(read_df)):
    answer = read_df.loc[i, "填写答案数字（改）"]
    initial_answer = read_df.loc[i, "填写答案数字"]
    if not np.isnan(answer):
        read_df.loc[i, "排序标识符"] = 0
    elif unknown(initial_answer):
        read_df.loc[i, "填写答案数字（改）"] = "不确定"
        read_df.loc[i, "填写答案数字（改+）"] = "不确定"
        read_df.loc[i, "排序标识符"] = 1
    if read_df.loc[i, "排序标识符"] == 0 and answer < 100:
        read_df.loc[i, "填写答案数字（改+）"] = answer * 10000   

In [58]:
read_df.sort_values(by="排序标识符", kind='mergesort', inplace=True)
read_df.reset_index(drop=True, inplace=True)

In [59]:
read_df.to_excel("/Users/gongchenyin/PycharmProjects/ra/data/201907/BQD_v1_result/test.xlsx")

In [60]:
# 修改率
len(read_df[read_df["排序标识符"] != 2])/len(read_df)

0.88

## 七、循环遍历所有excel，读取、处理并写入新文件夹

In [61]:
import os
filelist=[]
read_path = "/Users/gongchenyin/PycharmProjects/ra/data/201907/BQD_v1_test"
for root, dirs, files in os.walk(read_path):
    for name in files:
        file_name =os.path.join(root, name)
        if file_name.split('.')[-1]=='xlsx':
            filelist.append(file_name)

filelist

['/Users/gongchenyin/PycharmProjects/ra/data/201907/BQD_v1_test/213.xlsx',
 '/Users/gongchenyin/PycharmProjects/ra/data/201907/BQD_v1_test/136.xlsx',
 '/Users/gongchenyin/PycharmProjects/ra/data/201907/BQD_v1_test/177.xlsx',
 '/Users/gongchenyin/PycharmProjects/ra/data/201907/BQD_v1_test/209.xlsx',
 '/Users/gongchenyin/PycharmProjects/ra/data/201907/BQD_v1_test/208.xlsx',
 '/Users/gongchenyin/PycharmProjects/ra/data/201907/BQD_v1_test/176.xlsx',
 '/Users/gongchenyin/PycharmProjects/ra/data/201907/BQD_v1_test/212.xlsx',
 '/Users/gongchenyin/PycharmProjects/ra/data/201907/BQD_v1_test/242.xlsx',
 '/Users/gongchenyin/PycharmProjects/ra/data/201907/BQD_v1_test/214.xlsx',
 '/Users/gongchenyin/PycharmProjects/ra/data/201907/BQD_v1_test/173.xlsx',
 '/Users/gongchenyin/PycharmProjects/ra/data/201907/BQD_v1_test/241.xlsx',
 '/Users/gongchenyin/PycharmProjects/ra/data/201907/BQD_v1_test/133.xlsx',
 '/Users/gongchenyin/PycharmProjects/ra/data/201907/BQD_v1_test/172.xlsx',
 '/Users/gongchenyin/Pych

## 八、百分数类问题数据转换

In [62]:
valid_num_percent = [
        '〇', '一', '二', '三', '四', '五', '六', '七', '八', '九', '零',
        '壹', '贰', '叁', '肆', '伍', '陆', '柒', '捌', '玖', '貮', '两',
        '十', '拾'
    ]

In [63]:
valid_char_percent = [
        '来', '多', '几', '左', '吧', '以', '之', '内', '%'
    ]

In [64]:
re4 = '[^' + ''.join(valid_num_percent) + '\\d]+'
reObj4 = re.compile(re4)

In [65]:
re5a = '[0-9' + ''.join(valid_num_percent)  + ']+'
re5b = '[^0-9' + ''.join(valid_num_percent) + ''.join(valid_char_percent) + ' ]+'
re5 = re5a + re5b
reObj5 = re.compile(re5)

In [66]:
if reObj5.search("60 %"):
    print('符合')
else:
    print('不符合')

不符合


In [67]:
re6a = '[百分之0-9' + ''.join(valid_num_percent) + '%]+'
re6b = '[^百分之0-9' + ''.join(valid_num_percent) + ']+'
re6c = '[百分之0-9' + ''.join(valid_num_percent) + ']+'
re6 = re6a + re6b + re6c
reObj6 = re.compile(re6)

In [68]:
if reObj6.search("10% 10%"):
    print('符合')
else:
    print('不符合')

符合


In [69]:
def robust_clean_percent(num) -> str:
    # 先将num转化为str类型
    num = str(num)
    words = ["百分之百", "百分百", "百分之一百", "一百", "所有", "全部"]
    for word in words:
        if num.find(word) != -1:
            return '100'
    # 忽略多次出现数字字符的情形（空格也算分割点），如“1% 80%”、“118. 25%”、”200 0%“、“2o%”
    if reObj6.search(num):
        return ''
    # 忽略数字之后跟随非有效词缀的词/字，如“100号”，“一般”
    if reObj5.search(num):
        return ''
    result = reObj4.sub('', num)
    # 如果转换后的值大于100，返回空值
    if result.isdigit():
        if int(result) > 100:
            return ''
    return result

In [70]:
s11 = '10多'
s22 = robust_clean_percent(s11)
s33 = hybrid_to_arabic(s22)
print('最开始：', s11)
print('清洗后：', s22)
print('转换后：', s33)

最开始： 10多
清洗后： 10
转换后： 10


## 九、选择题答案量化

In [71]:
read_df.head(2)

,部门,姓名,编号,填写时间,问题1,填写答案数字（改+）,填写答案数字（改）,填写答案数字,问题2,填写答案,排序标识符
0,北京新双井店,杨秀斌,981101001,2019-05-22 00:00:00,您认为您所在门店上个月业绩最高的员工业绩(销售额)是多少,0,0,0,"您对您的答案有多确定\r\n选项：A. 很确定,B. 比较确定,C. 一般,D. 不太确定,...",B. 比较确定,0
1,即墨文化路店,梁军拴,170730018,2019-05-22 00:00:00,您认为您所在门店上个月业绩最高的员工业绩(销售额)是多少,0,0,0,"您对您的答案有多确定\r\n选项：A. 很确定,B. 比较确定,C. 一般,D. 不太确定,...",E. 很不确定,0


In [72]:
read_df["填写答案"].iloc[0].find("A")

-1

In [73]:
def quantify(answer: str) -> int:
    if answer.find("A") != -1:
        return 5
    elif answer.find("B") != -1:
        return 4
    elif answer.find("C") != -1:
        return 3
    elif answer.find("D") != -1:
        return 2
    elif answer.find("E") != -1:
        return 1
    elif answer.find("F") != -1:
        return 0
    else:
        return np.nan

In [74]:
for i in range(len(read_df.columns)):
    if read_df.columns[i] == "填写答案":
        index_of_answer = i
read_df.insert(index_of_answer, "填写答案（改）", read_df[["填写答案"]].applymap(quantify))

In [75]:
read_df.head()

,部门,姓名,编号,填写时间,问题1,填写答案数字（改+）,填写答案数字（改）,填写答案数字,问题2,填写答案（改）,填写答案,排序标识符
0,北京新双井店,杨秀斌,981101001,2019-05-22 00:00:00,您认为您所在门店上个月业绩最高的员工业绩(销售额)是多少,0,0,0,"您对您的答案有多确定\r\n选项：A. 很确定,B. 比较确定,C. 一般,D. 不太确定,...",4,B. 比较确定,0
1,即墨文化路店,梁军拴,170730018,2019-05-22 00:00:00,您认为您所在门店上个月业绩最高的员工业绩(销售额)是多少,0,0,0,"您对您的答案有多确定\r\n选项：A. 很确定,B. 比较确定,C. 一般,D. 不太确定,...",1,E. 很不确定,0
2,北京新双井店,刘少雄,190508045,2019-05-22 00:00:00,您认为您所在门店上个月业绩最高的员工业绩(销售额)是多少,0,0,0,"您对您的答案有多确定\r\n选项：A. 很确定,B. 比较确定,C. 一般,D. 不太确定,...",3,C. 一般,0
3,胶南灵山路店,王丹丹,170730052,2019-05-22 00:00:00,您认为您所在门店上个月业绩最高的员工业绩(销售额)是多少,0,0,0,"您对您的答案有多确定\r\n选项：A. 很确定,B. 比较确定,C. 一般,D. 不太确定,...",5,A. 很确定,0
4,黄岛桂冠店,宋茂利,180923012,2019-05-22 00:00:00,您认为您所在门店上个月业绩最高的员工业绩(销售额)是多少,10000,1,1,"您对您的答案有多确定\r\n选项：A. 很确定,B. 比较确定,C. 一般,D. 不太确定,...",5,A. 很确定,0
